## [PyTorch Inference Submission] EfficientNet Baseline From [original notebook](https://www.kaggle.com/rhtsingh/pytorch-training-inference-efficientnet-baseline)

*Note: I have exhausted my GPU for this week and so was unable to complete training. When training started I had only 1/30hr left.*


### Settup Dependencies

In [ ]:
# !pip install efficientnet_pytorch
# !pip install torch_optimizer

In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch-master/ > /dev/null # no output

In [ ]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import efficientnet_pytorch

# import torch_optimizer as optim
import torch.optim as optim
import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

### Train Configuration

*Note: Lots of improvement can be done simply here. e.g.*

* MIN SAMPLES PER CLASS - This variable is a threshold for total number of images in a class. If has class has less than this count then it will be discarded from training set.
* BATCH SIZE            - The number of images in each training batch.
* EPOCHS                - Total number of epochs.

In [ ]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
MIN_SAMPLES_PER_CLASS = 1 
BATCH_SIZE = 32
NUM_WORKERS = multiprocessing.cpu_count()
NUM_EPOCHS = 2
LOG_FREQ = 10
NUM_TOP_PREDICTS = 5
NUM_PUBLIC_TRAIN_IMAGES = 1580470

### Read Train and Test DataFrame

In [ ]:
# when re-running code, classes_num(num of landmark_ids) of the private train set  is less than the public train set, so we need to reload the original train.csv
public_train = pd.read_csv('../input/glr-train-csv/train.csv') # The public train.csv
train = pd.read_csv('../input/landmark-recognition-2020/train.csv')  # Treat as the private train.csv when submit to re-run code
test = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
train_dir = '../input/landmark-recognition-2020/train/'
test_dir = '../input/landmark-recognition-2020/test/'

> ### Dataset

image size=(3, 224, 224)

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        transforms_list = []
        if self.mode == 'train':
            # Increase image size from (64,64) to higher resolution,
            # Make sure to change in RandomResizedCrop as well.
            transforms_list = [
                transforms.Resize((224,224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(224),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
        else:
            transforms_list.extend([
                # Keep this resize same as train
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index: int):
        image_id = self.df.iloc[index].id
        image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
        image = Image.open(image_path)
        image = self.transforms(image)

        if self.mode == 'test':
            return {'image':image}
        else:
            return {'image':image, 
                    'target':self.df.iloc[index].landmark_id}

    def __len__(self) -> int:
        return self.df.shape[0]

### Load Data

In [ ]:
def load_data(train, test, train_dir, test_dir):
    counts = train.landmark_id.value_counts()
    selected_classes = counts[counts >= MIN_SAMPLES_PER_CLASS].index
    num_classes = selected_classes.shape[0]
    print('classes with at least N samples:', num_classes)

    train = train.loc[train.landmark_id.isin(selected_classes)]
    print('train_df', train.shape)
    print('test_df', test.shape)

    # filter non-existing test images
    exists = lambda img: os.path.exists(f'{test_dir}/{img[0]}/{img[1]}/{img[2]}/{img}.jpg')
    test = test.loc[test.id.apply(exists)]
    print('test_df after filtering', test.shape)

    label_encoder = LabelEncoder()
    label_encoder.fit(train.landmark_id.values)
    print('found classes', len(label_encoder.classes_))
    assert len(label_encoder.classes_) == num_classes

    train.landmark_id = label_encoder.transform(train.landmark_id)

    train_dataset = ImageDataset(train, train_dir, mode='train')
    test_dataset = ImageDataset(test, test_dir, mode='test')

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=False, num_workers=4, drop_last=True)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS)

    return train_loader, test_loader, label_encoder, num_classes, train.shape[0]

### Model

*Note: Used efficientnet-b0. Experimenting with different archs can yield different results*

In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes=81313):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        model_name = 'efficientnet-b' + str(self.depth)
#         self.base = efficientnet_pytorch.EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}')
#         self.base.load_state_dict(torch.load(efn_weights[model_name]))
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

### Inference Function

In [ ]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader, disable=IN_KERNEL)):
            if data_loader.dataset.mode != 'test':
                input_, target = data['image'], data['target']
            else:
                input_, target = data['image'], None

            output = model(input_.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

### Generate Submission

In [ ]:
def generate_submission(test_loader, model, label_encoder):
    sample_sub = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')

    predicts_gpu, confs_gpu, _ = inference(test_loader, model)
    predicts, confs = predicts_gpu.cpu().numpy(), confs_gpu.cpu().numpy()

    labels = [label_encoder.inverse_transform(pred) for pred in predicts]
    print('labels', np.array(labels).shape)
    print('confs', np.array(confs).shape)

    sub = test_loader.dataset.df
    
    def concat(label: np.ndarray, conf: np.ndarray) -> str:
#         result = ' '.join([f'{L} {c}' for L, c in zip(label, conf)])
        result = ''
        for L, c in zip(label, conf):
            if L in private_counts:
                result = f'{L} {c}'
                break
        
        return result
    
    sub['landmarks'] = [concat(label, conf) for label, conf in zip(labels, confs)]
#     sub['landmarks'] = [concat(label, conf) if label[0] in private_counts else '' for label, conf in zip(labels, confs)]

    sample_sub = sample_sub.set_index('id')
    sub = sub.set_index('id')
    sample_sub.update(sub)

    sample_sub.to_csv('submission.csv')

### Process

In [ ]:
if __name__ == '__main__':
    global_start_time = time.time()
    private_counts = train.landmark_id.value_counts()
    print('Private train set landmark_ids lenght:', len(private_counts))
#     train_loader, test_loader, label_encoder, num_classes, train_len = load_data(train, test, train_dir, test_dir)
    train_loader, test_loader, label_encoder, num_classes, train_len = load_data(public_train, test, train_dir, test_dir)
#     print(label_encoder.inverse_transform([100]))

    model = EfficientNetEncoderHead(depth=0, num_classes=num_classes)
    try:
        model_path = '../input/efnb0-10-gap05917pth/efn-b0_10_GAP0.5917.pth'
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        print('Model found in {}'.format(model_path))
    except:
        print('Random initialize model!')
        
    model.cuda()
#     print(model)

    print('inference mode')
    generate_submission(test_loader, model, label_encoder)

*Note: Will be publishing better kernels soon with more advanced techniques for landmark recognition.*
### More To Come. Stay Tuned. !!